In [ ]:
import sys
# sys.path.append("./") # go to parent dir
# sys.path.append("/Users/paulkreymborg/Documents/code/VMTutorial_pjk/2024_tests")
# print(sys.path)
import os
import pandas as pd
import plotly.express as px
import json
# import code
from viz_sim_results.visualize import make_plotly_visualizer
# import code.plot_cells.visualize
import numpy as np

# code.plot_cells
# print(dir(code))


In [ ]:
from simcode.theoretical_model.regular_hexagon_sympy_model import RegHexagonalModel
import math
from IPython.display import Markdown, display

battery_dir = "/Users/paulkreymborg/Documents/code/VMTutorial_pjk/scratch/bulkprop_battery"
# print(os.listdir(battery_dir))
with open(os.path.join(battery_dir, "manifest_experiments.json"), "r") as f:
    manifest_jobj = json.load(f)

experiment_idx = 4

with open(os.path.join(
    battery_dir,
    manifest_jobj["sub_experiments"][experiment_idx]["conf_path"],
), "r") as f:
    exp_conf = json.load(f)
    exp_params = exp_conf["params"]

print(exp_conf)

analytical_predictions = RegHexagonalModel().find_elastic_props_of_hexagon(
    A_0_num=exp_params["param_A0"],
    P_0_num=exp_params["param_P0"],
    K_num=exp_params["param_kappa"],
    gamma_num=exp_params["param_gamma"],
)
### Print some theoretical predictions
print(analytical_predictions)

field_over_shear_mod_ratio = exp_params["field_strength"]/analytical_predictions["shear_modulus"]

exp_shape_param = exp_params["param_P0"] / math.sqrt(exp_params["param_A0"])
# display(Markdown(r"## *$p_0$: {shape_param:.2f}, $E_{{field}}/B_{{shear}}$: {field_shearmod_ratio:.2f}, $\nu$: {nu:.2f}, $\gamma_{{friction}}$: {fric_gam:.2e}*".format(
#     shape_param =         exp_shape_param,
#     field_shearmod_ratio= field_over_shear_mod_ratio,
#     nu=                   analytical_predictions["poisson_ratio"],
#     fric_gam=           exp_params['vertex_friction_gamma'],
# )))
display(Markdown(r"## *$p_0$: {shape_param:.2f}, $E_{{field}}$: {field_strength:.2e}, $\nu$: {nu:.2f}, $\gamma_{{friction}}$: {fric_gam:.2e}*".format(
    shape_param =         exp_shape_param,
    field_strength= exp_params["field_strength"],
    nu=                   analytical_predictions["poisson_ratio"],
    fric_gam=           exp_params['vertex_friction_gamma'],
)))
display(Markdown(r"### $ B_{{shear}}={shear_mod:.2e}$".format(
    shear_mod=analytical_predictions["shear_modulus"],
)))
display(Markdown(r"### $ K_{{bulk}}={bulk_mod:.2e}$".format(
    bulk_mod=analytical_predictions["bulk_modulus"],
)))
display(Markdown(r"### $ \nu_{{poisson}}={poisson_rat:.4f}$".format(
    poisson_rat=analytical_predictions["poisson_ratio"],
)))

show_energy_terms = False

if show_energy_terms:
    display(Markdown(r"### $E_{cell}=\frac{K}{2}\left(A-A_0\right)^2+\frac{\Gamma}{2}\left(P-P_0\right)^2$"))
    display(Markdown(r"#### $A_0={A0:.4f}$".format(
        A0=exp_params["param_A0"]
    )))
    display(Markdown(r"#### $P_0={P0:.4f}$".format(
        P0=exp_params["param_P0"]
    )))
    display(Markdown(r"#### $K={kap:.4f}$".format(
        kap=exp_params["param_kappa"]
    )))
    display(Markdown(r"#### $\Gamma={gam:.4f}$".format(
        gam=exp_params["param_gamma"]
    )))

# print(manifest_jobj["sub_experiments"][experiment_idx])

# experiment_dir = os.path.join(ba"exp_8"
ckpts_dir = os.path.join(
    battery_dir,
    manifest_jobj["sub_experiments"][experiment_idx]["ckpts_dir"]
)
# ckpts_dir = os.path.join(experiment_dir, "ckpts")
tiss_ckpt_fps = []

include_every = 1
checkpoint_fns = list(sorted(os.listdir(ckpts_dir)))

if len(checkpoint_fns) == 0:
    raise Exception("No checkpoints found")
for ckpt_idx, fn in enumerate(checkpoint_fns[:]):
    # print(fn)
    if ckpt_idx % include_every != 0:
        continue
        
    if fn.endswith(".json") and fn.startswith("res"):
        tiss_ckpt_fps.append(os.path.join(ckpts_dir, fn))

fig = make_plotly_visualizer(
    tiss_ckpt_fps,
    os.path.join(ckpts_dir, "initial_vm_state.json"),
    # vertices_to_highlight=[0],
)

fig.show()


In [ ]:
# from tissue_builder.hexagonal import HexagonalCellMesh
# # import plotly.graph_objects as go
# # fig = go.Figure(go.Scatter(x=[1,3,None,2,4], y=[4,2,None,4,2],
# #             marker= dict(size=10,symbol= "arrow-bar-up", angleref="previous")))
# # fig.update_layout(width=500, height=300)
# # fig.show()


# list(init_vmstate['current_state']['forces'].keys())

In [ ]:
# # import json
# import shapely
# with open("../scratch/initial_vm_state.json", "r") as f:
#     init_vmstate = json.load(f)
    
# def plot_fields(vmstate):
#     elec_fields = []
#     for fid, fspec in vmstate['current_state']['forces'].items():
#         if fspec['type'] == "electric_cell_boundary_force":
#             elec_fields.append(fspec['field_spec'])
#     print(elec_fields)
#     for e_f_spec in elec_fields:
#         poly_bounds = shapely.Polygon(e_f_spec["zone_bounds"]["polygon_vertices"])
#         zone_center = shapely.centroid(poly_bounds)
#         print(zone_center.x)

# plot_fields(init_vmstate)
# # # cm = HexagonalCellMesh(
# # #     side_length=1.0,
# # #     box_lx=3,
# # #     box_ly=3,
# # # )
# # # cm.set_all_A0(1)
# # # cm.set_all_P0(1)
# # # cm.build_vm_mesh("../scratch/nb_example.json")

# # with open("../scratch/example.json") as f:
# #     exam_obj = json.load(f)

# # # exam_obj